In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import random_split
from torchtext.vocab import build_vocab_from_iterator
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
tkDF = pd.read_csv('../data/경상도.csv')

In [42]:
tkDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212906 entries, 0 to 212905
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   사투리     212906 non-null  object
 1   표준어     212906 non-null  object
dtypes: object(2)
memory usage: 3.2+ MB


In [43]:
tkDF.head(20)

,사투리,표준어
0,지난달 그 지난달에는 그의 거기가 어딘지 내가 그곳을 말을 모르는데 그 정신이 들에...,지난달 그 지난달에는 그의 거기가 어딘지 내가 그곳을 말을 모르는데 그 정신이 들에...
1,힘들어요 농사 짓는 거는 힘들고 요즘은 기계가 좋아 가지고 농사 짓기가 수월하지만도...,힘들어요 농사 짓는 거는 힘들고 요즘은 기계가 좋아 가지고 농사 짓기가 수월하지만 ...
2,더운 삼계탕 겉은 거는 더울 때 묵을 때는 좋지예 좋은데 묵고 나면 사람이 속도 편...,더운 삼계탕 같은 거는 더울 때 먹을 때는 좋지요 좋은데 먹고 나면 사람이 속도 편...
3,장 보러 가다가 비가 마이 오길래 밭두럼 엉개질까봐 밭에 가보는 길이시더,장 보러 가다가 비가 많이 오길래 밭두렁 무너질까봐 밭에 가보는 길입니다
4,비가 오니까 집에 어데 누전이 되는가 그건 한번 되더라고예 그래 정전될 때는 인자 ...,비가 오니까 집에 어디 누전이 되는가 그건 한번 되더라구요 그래 정전될 때는 인제 ...
5,우리 집에서 불편한 점은 없는데 아 그기 저기 기차역이 지나가니까 시끄러워 가지고 ...,우리 집에서 불편한 점은 없는데 아 거기 저기 기차역이 지나가니까 시끄러워 가지고 ...
6,부추는 부산 말로 정구지 아이가 정구지는 지짐을 해 먹으면은 제일 좋은데 그게 비가...,부추는 부산 말로 부추 아닌가요 부추는 전을 해 먹으면은 제일 좋은데 그게 비가 좀...
7,이 구두 하나만 계속 신고 다니니께 인자 굽이 딿아서 갈아야 되겠네,이 구두 하나만 계속 신고 다니니까 이제 굽이 닳아서 갈아야 되겠네
8,부추 우리 말로 정구지라고 하는데 정구지는 딱 보면은 부침개 해가지고 정구지 전 구...,부추 우리 말로 부추라고 하는데 부추는 딱 보면은 부침개 해가지고 부추 전 구워가지...
9,요즘 몸에 좋은 거는 별 보조식품은 없는데 인제 주로 버섯 같은 거 좀 많이 먹고 ...,요즘 몸에 좋은 거는 별 보조식품은 없는데 인제 주로 버섯 같은 거 좀 많이 먹고 ...


In [44]:
tkDF = pd.melt(tkDF)

In [45]:
tkDF = tkDF.sample(frac=1).reset_index(drop=True)

In [46]:
# 중복값 제거
tkDF.drop_duplicates(inplace=True)

In [47]:
tkDF['variable'].values

array(['사투리', '표준어', '사투리', ..., '사투리', '표준어', '표준어'], dtype=object)

In [48]:
import string

tkDF['value'] = tkDF['value'].replace(r'[{}]'.format(string.punctuation), '', regex=True)

In [49]:
tkDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 308433 entries, 0 to 425811
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   variable  308433 non-null  object
 1   value     308433 non-null  object
dtypes: object(2)
memory usage: 7.1+ MB


In [50]:
class dataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.DF = df
        self.label = [1 if value == '사투리' else 0 for value in self.DF['variable'].values]
        self.text = self.DF['value'].values

    def __len__(self): 
        return len(self.text)
    
    def __getitem__(self, idx):
        return self.label[idx],self.text[idx]

In [51]:
tkDS = dataset(tkDF)

In [52]:
train_length = int(0.8 * len(tkDS))
val_length = len(tkDS) - train_length
trainDS, valDS = random_split(tkDS, [train_length, val_length])

In [53]:
# 토큰 관련 특별 문자
unk = '<UNK>'
pad = '<PAD>'

In [54]:
from konlpy.tag import Mecab
mecab = Mecab()

In [55]:
# 한글 불용어 제거 # 어근 제거는 안함 - 사투리와 표준어 구별 사라짐
def torkenizer(text):
    with open('data/hangul_stopword.txt', 'r', encoding='utf-8') as f:
        stopword_h = {line.strip() for line in f.readlines()} 
    token = mecab.morphs(text)
    token = [word for word in token if word not in stopword_h] 
    return token


In [56]:
def yield_tokens(data_iter):
    for label, text in data_iter:
        yield torkenizer(text)

In [57]:
# !pip3 install mecab-ko-msvc mecab-ko-dic-msvc

In [58]:
VOCAB = build_vocab_from_iterator(yield_tokens(trainDS), specials=[unk, pad], special_first=True)

VOCAB.set_default_index(VOCAB[unk])

In [59]:
# 텍스트 > 정수 인코딩
text_pipeline = lambda x: VOCAB(torkenizer(x))

In [60]:
# 인코딩
token2id = {label: id for id, label in enumerate(VOCAB.get_itos())}
# 디코딩
id2token = {id: label for id, label in enumerate(VOCAB.get_itos())}

In [61]:
# 배치 크기 만큼 데이터셋 반환 함수
def collate_batch(batch):

    label_list, text_list, offsets = [], [], [0] 

    for (_label, _text) in batch:

        label_list.append(_label)

        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)

        offsets.append(processed_text.size(0))

    # 텐서화 진행
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0) # 누적값 -> 전체 갯수
    text_list = torch.cat(text_list)
    
    return label_list.to(DEVICE), text_list.to(DEVICE), offsets.to(DEVICE)

In [62]:
BATCH_SIZE = 64
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE, collate_fn=collate_batch)
testDL = DataLoader(valDS, batch_size=BATCH_SIZE, collate_fn=collate_batch)

In [63]:
NUM_CLASSE = 1
VOCAB_SIZE = len(VOCAB)
NUM_LAYER = 1
print(f'NUM_CLASSES: {NUM_CLASSE} VOCAB_SIZE: {VOCAB_SIZE}')

NUM_CLASSES: 1 VOCAB_SIZE: 47562


In [64]:
class TextModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_class):
        super(TextModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, sparse=False)
        self.rnn = nn.GRU(embedding_dim, hidden_size, NUM_LAYER, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self,text, offsets):
        embedded = self.embedding(text, offsets)
        output, hidden = self.rnn(embedded)
        return self.fc(output) 

In [65]:
HIDDEN_SIZE = 3
EMBEDDING_DIM = 64
VOCAB_SIZE = len(VOCAB)
EPOCHS = 10
LR = 0.1

In [66]:
# 학습 관련 인스턴스
MODEL = TextModel(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_SIZE, NUM_CLASSE).to(DEVICE)

CRITERION = nn.BCEWithLogitsLoss()
OPTIMIZER = torch.optim.Adam(MODEL.parameters(), lr=LR)
SCHEDULER = torch.optim.lr_scheduler.ReduceLROnPlateau(OPTIMIZER)

In [67]:
from torchmetrics.classification import BinaryF1Score

In [68]:
def train(model, DL, loss_fn, optimizer):
    model.train()
    lossList = []

    f1 = BinaryF1Score()

    for label, text, offset in DL: 
        label = label.to(DEVICE)
        text = text.to(DEVICE)  
        offset = offset.to(DEVICE)
        
        output = model(text, offset)

        target = label.unsqueeze(1).float()  # [배치 크기, 1] 형태로 변환
        
        loss = loss_fn(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lossList.append(loss.item())
        
        f1(output, target)

    train_loss = np.mean(lossList)
    train_f1 = f1.compute().cpu().item()

    print(f'[Train loss] ==> {train_loss:.4f}    [Train Accuracy] ==> {train_f1:4f}')
    return train_loss, train_f1

In [69]:
def evaluate(model, DL, loss_fn):
    model.eval()
    losses = []
    
    f1 = BinaryF1Score()

    with torch.no_grad():
        for label, text, offset in DL: 
            label = label.to(DEVICE)
            text = text.to(DEVICE)
            offset = offset.to(DEVICE)
            
            output = model(text, offset)

            
            target = label.unsqueeze(1).float()  # [배치 크기, 1] 형태로 변환
            
            loss = loss_fn(output, target)
            losses.append(loss.item())

            f1(output, target)
        
    val_loss = np.mean(losses)
    val_f1 = f1.compute().cpu().item()

    print(f'[Valid loss] ==> {val_loss:4f}    [Valid Accuracy] ==> {val_f1:4f}')
    return val_loss, val_f1

In [70]:
def predict(model, text, text_pipeline):
    with torch.no_grad():
        # 토큰화 > 정수 변환 > 텐서
        text = torch.tensor(text_pipeline(text), dtype=torch.int64).to(DEVICE)
        text = text.unsqueeze(0)
        offsets = torch.tensor([0]).DEVICE       
        predicted_label = model(text, offsets)
        return predicted_label.argmax(1).item() + 1

In [71]:
# 학습 및 검증 진행
train_ = [[],[]]
test_ = [[],[]]

min_loss = float('inf')

for epoch in range(1, EPOCHS+1):
    print(f'[Epoch {epoch}/{EPOCHS}]')
    train_loss, train_f1 = train(MODEL, trainDL, CRITERION, OPTIMIZER)
    val_loss, val_f1 = evaluate(MODEL, testDL, CRITERION)

    train_[0].append(train_loss)
    train_[1].append(train_f1)
    train_[0].append(train_loss)
    train_[1].append(train_f1)

    SCHEDULER.step(val_loss)

    if min_loss > val_loss:
        min_loss = val_loss
        torch.save(MODEL, 'model.tkLang.pt')
    if SCHEDULER.num_bad_epochs >= SCHEDULER.patience:
        print(f'Early Stopping at {epoch}')
        torch.save(MODEL, 'model.tkLang.pt')
        break

[Epoch 1/10]
[Train loss] ==> 0.5072    [Train Accuracy] ==> 0.747324
[Valid loss] ==> 0.499009    [Valid Accuracy] ==> 0.733524
[Epoch 2/10]
[Train loss] ==> 0.4928    [Train Accuracy] ==> 0.762998
[Valid loss] ==> 0.490618    [Valid Accuracy] ==> 0.740987
[Epoch 3/10]
[Train loss] ==> 0.4807    [Train Accuracy] ==> 0.768460
[Valid loss] ==> 0.481046    [Valid Accuracy] ==> 0.762715
[Epoch 4/10]
[Train loss] ==> 0.4751    [Train Accuracy] ==> 0.765316
[Valid loss] ==> 0.489634    [Valid Accuracy] ==> 0.768920
[Epoch 5/10]


KeyboardInterrupt: 